<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


## Final Project - Build an ML Pipeline for Airfoil noise prediction


Estimated time needed: **90** minutes


## Scenario


You are a data engineer at an aeronautics consulting company. Your company prides itself in being able to efficiently design airfoils for use in planes and sports cars. Data scientists in your office need to work with different algorithms and data in different formats. While they are good at Machine Learning, they count on you to be able to do ETL jobs and build ML pipelines. In this project you will use the modified version of the NASA Airfoil Self Noise dataset. You will clean this dataset, by dropping the duplicate rows, and removing the rows with null values. You will create an ML pipe line to create a model that will predict the SoundLevel based on all the other columns. You will evaluate the model and towards the end you will persist the model.



## Objectives

In this 4 part assignment you will:

- Part 1 Perform ETL activity
  - Load a csv dataset
  - Remove duplicates if any
  - Drop rows with null values if any
  - Make transformations
  - Store the cleaned data in parquet format
- Part 2 Create a  Machine Learning Pipeline
  - Create a machine learning pipeline for prediction
- Part 3 Evaluate the Model
  - Evaluate the model using relevant metrics
- Part 4 Persist the Model 
  - Save the model for future production use
  - Load and verify the stored model


## Datasets

In this lab you will be using dataset(s):

 - The original dataset can be found here NASA airfoil self noise dataset. https://archive.ics.uci.edu/dataset/291/airfoil+self+noise
 
 - This dataset is licensed under a Creative Commons Attribution 4.0 International (CC BY 4.0) license.


Diagram of an airfoil. - For informational purpose


![Airfoil with flow](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_with_flow.png)


Diagram showing the Angle of attack. - For informational purpose


![Airfoil angle of attack](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/images/Airfoil_angle_of_attack.jpg)


## Before you Start


**Before you start attempting this project it is highly recommended that you finish the practice project.**


## Setup


For this lab, we will be using the following libraries:

*   [`PySpark`](https://spark.apache.org/docs/latest/api/python/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01) for connecting to the Spark Cluster


### Installing Required Libraries

Spark Cluster is pre-installed in the Skills Network Labs environment. However, you need libraries like pyspark and findspark to
 connect to this cluster.


The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [8]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [9]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

## Part 1 - Perform ETL activity


### Task 1 - Import required libraries


In [10]:
#your code goes here
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
import pandas as pd

from pyspark.sql.functions import col, expr,upper,udf,length,size,split,concat,regexp_replace,row_number,count
from pyspark.sql.types import DoubleType,StringType
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, CountVectorizer, IDF
from pyspark.ml import Pipeline
from pyspark.sql.window import Window


### Task 2 - Create a spark session


In [11]:
#Create a SparkSession
# Initialize Spark session

spark = SparkSession.builder.appName("Practice Project").getOrCreate()


### Task 3 - Load the csv file into a dataframe


Download the data file.

NOTE : Please ensure you use the dataset below and not the original dataset mentioned above.


In [12]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv


--2024-10-14 05:56:38--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60682 (59K) [text/csv]
Saving to: ‘NASA_airfoil_noise_raw.csv.2’

NASA_airfoil_noise_ 100%[===================>]  59.26K  --.-KB/s    in 0.001s  

2024-10-14 05:56:38 (70.2 MB/s) - ‘NASA_airfoil_noise_raw.csv.2’ saved [60682/60682]



Load the dataset into the spark dataframe


In [23]:
# Load the dataset that you have downloaded in the previous task

df = spark.read.csv('NASA_airfoil_noise_raw.csv',header = True, inferSchema=True)
df.show()

+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|      800|          0.0|     0.3048|              71.3|             0.00266337|   126.201|
|     1000|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     1250|          0.0|     0.3048|              71.3|             0.00266337|   125.951|
|     1600|          0.0|     0.3048|              71.3|             0.00266337|   127.591|
|     2000|          0.0|     0.3048|              71.3|             0.00266337|   127.461|
|     2500|          0.0|     0.3048|              71.3|             0.00266337|   125.571|
|     3150|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     4000|          0.0|     0.3048|              71.3|             0.00266337|

### Task 4 - Print top 5 rows of the dataset


In [24]:
#your code goes here
df.show(5)

+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|      800|          0.0|     0.3048|              71.3|             0.00266337|   126.201|
|     1000|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     1250|          0.0|     0.3048|              71.3|             0.00266337|   125.951|
|     1600|          0.0|     0.3048|              71.3|             0.00266337|   127.591|
|     2000|          0.0|     0.3048|              71.3|             0.00266337|   127.461|
+---------+-------------+-----------+------------------+-----------------------+----------+
only showing top 5 rows



### Task 6 - Print the total number of rows in the dataset


In [25]:
#your code goes here
rowcount1 = df.count()
print(rowcount1)

1522


### Task 7 - Drop all the duplicate rows from the dataset


In [26]:
df = df.dropDuplicates()#


### Task 8 - Print the total number of rows in the dataset


In [27]:
#your code goes here

rowcount2 =df.count()
print(rowcount2)


[Stage 26:============================================>         (163 + 9) / 200]

1503


### Task 9 - Drop all the rows that contain null values from the dataset


In [28]:
df = df.dropna()


### Task 10 - Print the total number of rows in the dataset


In [29]:
#your code goes here

rowcount3 = df.count()
print(rowcount3)


[Stage 29:==========================================>           (157 + 8) / 200]

1499


### Task 11 - Rename the column "SoundLevel" to "SoundLevelDecibels"


In [30]:
# your code goes here

df = df.withColumn('SoundLevelDecibels',df['SoundLevel'])
df = df.drop('SoundLevel')
df.show()

+---------+-------------+-----------+------------------+-----------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|
+---------+-------------+-----------+------------------+-----------------------+------------------+
|     4000|          3.0|     0.3048|              31.7|             0.00529514|           115.608|
|     3150|          2.0|     0.2286|              31.7|             0.00372371|           121.527|
|     2000|          7.3|     0.2286|              31.7|              0.0132672|           115.309|
|     2000|          5.4|     0.1524|              71.3|             0.00401199|           131.111|
|      500|          9.9|     0.1524|              71.3|              0.0193001|           131.279|
|     1000|         12.6|     0.1524|              71.3|              0.0483159|           122.044|
|    12500|          0.0|     0.0254|              39.6|             4.28464E-4|           129.116|


### Task 12 - Save the dataframe in parquet format, name the file as "NASA_airfoil_noise_cleaned.parquet"


In [31]:
# your code goes here
df.write.mode("overwrite").parquet("NASA_airfoil_noise_cleaned.parquet")
#df.write.parquet("NASA_airfoil_noise_cleaned2.parquet")


#### Part 1 - Evaluation



**Run the code cell below.**<br>
**Use the answers here to answer the final evaluation quiz in the next section.**<br>
**If the code throws up any errors, go back and review the code you have written.**</b>


In [32]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("New column name = ", df.columns[-1])

import os

print("NASA_airfoil_noise_cleaned.parquet exists :", os.path.isdir("NASA_airfoil_noise_cleaned.parquet"))

Part 1 - Evaluation
Total rows =  1522
Total rows after dropping duplicate rows =  1503
Total rows after dropping duplicate rows and rows with null values =  1499
New column name =  SoundLevelDecibels
NASA_airfoil_noise_cleaned.parquet exists : True


## Part - 2 Create a  Machine Learning Pipeline


### Task 1 - Load data from "NASA_airfoil_noise_cleaned.parquet" into a dataframe


In [33]:
#your code goes here

df = spark.read.parquet('NASA_airfoil_noise_cleaned.parquet')


### Task 2 - Print the total number of rows in the dataset


In [34]:
df.show()

+---------+-------------+-----------+------------------+-----------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|
+---------+-------------+-----------+------------------+-----------------------+------------------+
|      630|          0.0|     0.3048|              31.7|             0.00331266|           129.095|
|     4000|          0.0|     0.3048|              31.7|             0.00331266|           118.145|
|     4000|          1.5|     0.3048|              39.6|             0.00392107|           117.741|
|      800|          4.0|     0.3048|              71.3|             0.00497773|           131.755|
|     1250|          0.0|     0.2286|              31.7|              0.0027238|           128.805|
|     2500|          4.0|     0.2286|              55.5|              0.0042862|           122.384|
|     1250|          7.3|     0.2286|              71.3|              0.0104404|           127.358|


In [35]:
#your code goes here

rowcount4 = df.count()
print(rowcount4)



[Stage 39:>                                                         (0 + 8) / 8]

1499


### Task 3 - Define the VectorAssembler pipeline stage


Stage 1 - Assemble the input columns into a single column "features". Use all the columns except SoundLevelDecibels as input features.


In [ ]:
df=df.withColumn('Frequency',df['Frequency'].cast("float").alias('Frequency'))
df=df.withColumn('AngleOfAttack',df['AngleOfAttack'].cast("float").alias('AngleOfAttack'))
df=df.withColumn('ChordLength',df['ChordLength'].cast("float").alias('ChordLength'))
df=df.withColumn('FreeStreamVelocity',df['FreeStreamVelocity'].cast("float").alias('FreeStreamVelocity'))
df=df.withColumn('SuctionSideDisplacement',df['SuctionSideDisplacement'].cast("float").alias('SuctionSideDisplacement'))
df=df.withColumn('SoundLevelDecibels',df['SoundLevelDecibels'].cast("float").alias('SoundLevelDecibels'))


In [36]:
#your code goes here
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

assembler = VectorAssembler(
    inputCols=["Frequency", "AngleOfAttack", "ChordLength",'FreeStreamVelocity','SuctionSideDisplacement'],
    outputCol="mmm")

#output = assembler.transform(df)
#output.select( "marks").show(truncate=False)

In [37]:
df.show()

+---------+-------------+-----------+------------------+-----------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|
+---------+-------------+-----------+------------------+-----------------------+------------------+
|      630|          0.0|     0.3048|              31.7|             0.00331266|           129.095|
|     4000|          0.0|     0.3048|              31.7|             0.00331266|           118.145|
|     4000|          1.5|     0.3048|              39.6|             0.00392107|           117.741|
|      800|          4.0|     0.3048|              71.3|             0.00497773|           131.755|
|     1250|          0.0|     0.2286|              31.7|              0.0027238|           128.805|
|     2500|          4.0|     0.2286|              55.5|              0.0042862|           122.384|
|     1250|          7.3|     0.2286|              71.3|              0.0104404|           127.358|


### Task 4 - Define the StandardScaler pipeline stage


Stage 2 - Scale the "features" using standard scaler and store in "scaledFeatures" column


In [38]:
#your code goes here
from pyspark.ml.feature import MinMaxScaler, StandardScaler

scaler = StandardScaler(inputCol="mmm", outputCol="sss")

#scaler_model = scaler.fit(output.select("marks"))
#sdf_scaled = scaler_model.transform(output)
#sdf_scaled.show()


### Task 5 - Define the Model creation pipeline stage


Stage 3 - Create a LinearRegression stage to predict "SoundLevelDecibels"

**Note:You need to use the scaledfeatures retreived in the previous step(StandardScaler pipeline stage).**


In [39]:
#your code goes here
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol="sss",labelCol="SoundLevelDecibels",  predictionCol="predicted_sd")


### Task 6 - Build the pipeline


Build a pipeline using the above three stages


In [40]:
#your code goes here
from pyspark.ml import Pipeline

pipelinenew = Pipeline(stages=[assembler, scaler, lr])


### Task 7 - Split the data


In [41]:
# Split the data into training and testing sets with 70:30 split.
# set the value of seed to 42
# the above step is very important. DO NOT set the value of seed to any other value other than 42.

#your code goes here

(trainingData, testingData) = df.randomSplit(weights=[0.7,0.3], seed=42)




### Task 8 - Fit the pipeline


In [42]:
# Fit the pipeline using the training data
# your code goes here

pipelineModel = pipelinenew.fit(trainingData)


24/10/14 06:02:05 WARN util.Instrumentation: [ab2decaa] regParam is zero, which might cause numerical instability and overfitting.
[Stage 45:>                                                         (0 + 8) / 8]24/10/14 06:02:08 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/10/14 06:02:08 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/10/14 06:02:08 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/10/14 06:02:08 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


#### Part 2 - Evaluation



**Run the code cell below.**<br>
**Use the answers here to answer the final evaluation quiz in the next section.**<br>
**If the code throws up any errors, go back and review the code you have written.**</b>


In [43]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipelinenew.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  1499
Pipeline Stage 1 =  VectorAssembler
Pipeline Stage 2 =  StandardScaler
Pipeline Stage 3 =  LinearRegression
Label column =  SoundLevelDecibels


## Part 3 - Evaluate the Model


### Task 1 - Predict using the model


In [44]:
# Make predictions on testing data
# your code goes here

predictions  = pipelineModel.transform(testingData)



In [45]:
predictions.show()

[Stage 52:>                                                         (0 + 1) / 1]

+---------+-------------+-----------+------------------+-----------------------+------------------+--------------------+--------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|                 mmm|                 sss|      predicted_sd|
+---------+-------------+-----------+------------------+-----------------------+------------------+--------------------+--------------------+------------------+
|      200|          9.9|     0.1524|              39.6|              0.0233328|           127.315|[200.0,9.9,0.1524...|[0.06338939572741...|123.64344009624753|
|      200|         15.4|     0.0508|              31.7|              0.0289853|           119.975|[200.0,15.4,0.050...|[0.06338939572741...|123.48695788614877|
|      200|         15.4|     0.0508|              39.6|              0.0282593|           121.783|[200.0,15.4,0.050...|[0.06338939572741...|124.38983849684254|
|      250|         19.7|     0.05

### Task 2 - Print the MSE


In [46]:
#your code goes here
from pyspark.mllib.evaluation import RegressionMetrics
valuesAndPreds = predictions.select(['SoundLevelDecibels', 'predicted_sd'])
# It needs to convert to RDD as the parameter of RegressionMetrics
valuesAndPreds = valuesAndPreds.rdd.map(tuple)


metrics = RegressionMetrics(valuesAndPreds)
#TODO
#TODO
mse = metrics.meanSquaredError
print(mse)


[Stage 53:=============================>                            (4 + 4) / 8]

22.593754071348812


In [47]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="predicted_sd", labelCol="SoundLevelDecibels", metricName="mse")
mse = evaluator.evaluate(predictions)

### Task 3 - Print the MAE


In [48]:
#your code goes here

#TODO
mae = metrics.meanAbsoluteError
print(mae)


3.7336902294631287


In [49]:
evaluator = RegressionEvaluator(predictionCol="predicted_sd", labelCol="SoundLevelDecibels", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)

[Stage 57:>                                                         (0 + 8) / 8]

3.7336902294631287


### Task 4 - Print the R-Squared(R2)


In [50]:
#your code goes here

#TODO
r2 = metrics.r2
print(r2)


0.06109667981825406


In [51]:
evaluator = RegressionEvaluator(predictionCol="predicted_sd", labelCol="SoundLevelDecibels", metricName="r2")
r2 = evaluator.evaluate(predictions)
print(r2)

[Stage 59:>                                                         (0 + 8) / 8]

0.5426016508689058


#### Part 3 - Evaluation



**Run the code cell below.**<br>
**Use the answers here to answer the final evaluation quiz in the next section.**<br>
**If the code throws up any errors, go back and review the code you have written.**</b>


In [52]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

#lrModel = pipelineModel.stages[-1]

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))

Part 3 - Evaluation
Mean Squared Error =  22.59
Mean Absolute Error =  3.73
R Squared =  0.54
Intercept =  132.6


## Part 4 - Persist the Model


### Task 1 - Save the model to the path "Final_Project"


In [53]:
# Save the pipeline model as "Final_Project"
# your code goes here
#mPath =  "labs/authoride/IBMSkillsNetwork+BD0231EN/labs/"
#pipelineModel.write().save("aaaaaaaaa")
pipelineModel.write().overwrite().save("bbbbbb")


### Task 2 - Load the model from the path "Final_Project"


In [54]:
# Load the pipeline model you have created in the previous step
loadedPipelineModel = pipelineModel.load("bbbbbb")


### Task 3 - Make predictions using the loaded model on the testdata


In [56]:
# Use the loaded pipeline model and make predictions using testingData
predictions  = pipelineModel.transform(testingData)


### Task 4 - Show the predictions


In [57]:
#show top 5 rows from the predections dataframe. Display only the label column and predictions
#your code goes here
predictions.show()

[Stage 81:>                                                         (0 + 1) / 1]

+---------+-------------+-----------+------------------+-----------------------+------------------+--------------------+--------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|                 mmm|                 sss|      predicted_sd|
+---------+-------------+-----------+------------------+-----------------------+------------------+--------------------+--------------------+------------------+
|      200|          9.9|     0.1524|              39.6|              0.0233328|           127.315|[200.0,9.9,0.1524...|[0.06338939572741...|123.64344009624753|
|      200|         15.4|     0.0508|              31.7|              0.0289853|           119.975|[200.0,15.4,0.050...|[0.06338939572741...|123.48695788614877|
|      200|         15.4|     0.0508|              39.6|              0.0282593|           121.783|[200.0,15.4,0.050...|[0.06338939572741...|124.38983849684254|
|      250|         19.7|     0.05

#### Part 4 - Evaluation




**Run the code cell below.**<br>
**Use the answers here to answer the final evaluation quiz in the next section.**<br>
**If the code throws up any errors, go back and review the code you have written.**</b>


In [60]:
print("Part 4 - Evaluation")

loadedmodel = loadedPipelineModel.stages[-1]
totalstages = len(loadedPipelineModel.stages)
inputcolumns = loadedPipelineModel.stages[0].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Part 4 - Evaluation
Number of stages in the pipeline =  3
Coefficient for Frequency is -3.9728
Coefficient for AngleOfAttack is -2.4775
Coefficient for ChordLength is -3.3818
Coefficient for FreeStreamVelocity is 1.5789
Coefficient for SuctionSideDisplacement is -1.6465


In [62]:
print(loadedmodel.coefficients)
print([round(i,4) for i in loadedmodel.coefficients ])

[-3.9728079740154767,-2.477510117497856,-3.3817506717710835,1.5789334478964052,-1.6465101501795425]
[-3.9728, -2.4775, -3.3818, 1.5789, -1.6465]


### Stop Spark Session


In [63]:
spark.stop()

## Authors


[Ramesh Sannareddy](https://www.linkedin.com/in/rsannareddy/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMBD0231ENSkillsNetwork866-2023-01-01)


### Other Contributors


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-05-26|0.1|Ramesh Sannareddy|Initial Version Created|


Copyright © 2023 IBM Corporation. All rights reserved.
